In [ ]:
import os
import sys
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Define and set the working directory
new_directory = '/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni'
if os.path.isdir(new_directory):
    os.chdir(new_directory)
    sys.path.append(new_directory)
    print("Current directory:", os.getcwd())
else:
    print("Directory does not exist:", new_directory)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Current directory: /content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni


In [ ]:
!pip install h3==3.* geopandas shapely matplotlib osmnx

# Current Scenario

## Load and Display Merged Dataset

In [ ]:
import pandas as pd  # Import the pandas library for data manipulation

# Load the CSV file containing the current data on hexagons
df_attuale = pd.read_csv('df_attuale_esagoni.csv')

# Clean the 'Comune' column: remove leading/trailing spaces and convert to lowercase
df_attuale['Comune'] = df_attuale['Comune'].str.strip().str.lower()

# Display the first few rows of the DataFrame
df_attuale.head()

# Load the Excel file (commented out for now)
# df_brescia = pd.read_excel('/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/Stime + Previsioni/dataset_unito.xlsx')

# Print the first few rows of the DataFrame to the console
print(df_attuale.head())

                 Comune  Pop_tot  Quota Popolazione  EV Stimati (Attuali)  \
0               brescia   197236           0.156869           1568.694197   
1   desenzano del garda    29197           0.023222            232.215034   
2           montichiari    26180           0.020822            208.219666   
3             lumezzane    21501           0.017101            171.005769   
4  palazzolo sull'oglio    20163           0.016036            160.364138   

   Domanda Mensile (kWh) - Attuali  Domanda Annua (kWh) - Attuali  \
0                    282364.955469                   3.388379e+06   
1                     41798.706143                   5.015845e+05   
2                     37479.539912                   4.497545e+05   
3                     30781.038490                   3.693725e+05   
4                     28865.544815                   3.463865e+05   

   Domanda Giornaliera (kWh) - Attuali  Colonnine Necessarie - Attuali  \
0                          9283.231413          

In [ ]:
# Assign df_attuale to df_finale (create a working copy or alias)
df_finale = df_attuale

# Count the number of unique municipalities in the 'Comune' column
numero_comuni = df_finale['Comune'].nunique()

# Print the number of distinct municipalities
print(f"Numero di comuni distinti: {numero_comuni}")

# Display the first few rows of the final DataFrame
print(df_finale.head())

Numero di comuni distinti: 205
                 Comune  Pop_tot  Quota Popolazione  EV Stimati (Attuali)  \
0               brescia   197236           0.156869           1568.694197   
1   desenzano del garda    29197           0.023222            232.215034   
2           montichiari    26180           0.020822            208.219666   
3             lumezzane    21501           0.017101            171.005769   
4  palazzolo sull'oglio    20163           0.016036            160.364138   

   Domanda Mensile (kWh) - Attuali  Domanda Annua (kWh) - Attuali  \
0                    282364.955469                   3.388379e+06   
1                     41798.706143                   5.015845e+05   
2                     37479.539912                   4.497545e+05   
3                     30781.038490                   3.693725e+05   
4                     28865.544815                   3.463865e+05   

   Domanda Giornaliera (kWh) - Attuali  Colonnine Necessarie - Attuali  \
0                

## Loading a Brescia province dataset for specific columns

In [ ]:
# Load the CSV file containing data for municipalities in Brescia
df_brescia = pd.read_csv("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/df_brescia.csv")

# Clean the 'Comune' column: remove leading/trailing spaces and convert to lowercase
df_brescia['Comune'] = df_brescia['Comune'].str.strip().str.lower()

# Display the first few rows of the DataFrame
df_brescia.head()

,Comune,Popolazione,Superficie,Densità,Altitudine
0,brescia,199949,90.58,2207.0,149
1,desenzano del garda,29275,59.19,495.0,67
2,montichiari,26372,82.02,322.0,104
3,lumezzane,21609,31.39,689.0,460
4,palazzolo sull'oglio,20390,23.31,875.0,166


## Creating indices


### Importing libraries and setting resolution

This step imports necessary Python libraries and also sets the H3 resolution level to 8 (moderate granularity).

In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import h3
from shapely.geometry import Polygon

# Set the resolution level for H3 hexagons (higher = finer grid)
resolution = 8

### Load boundaries and municipalities

In [ ]:
# Use df_attuale as the working DataFrame
df_finale = df_attuale

# Get the geometry (polygon) of the province of Brescia
provincia_gdf = ox.geocode_to_gdf("Brescia, Italy")
brescia_polygon = provincia_gdf.geometry.iloc[0]

# Extract the list of municipalities (comuni)
comuni = df_finale['Comune'].tolist()

# Initialize an empty list to store the geometries of each comune
comuni_polygons = []

# Geocode each comune to get its polygon geometry
for comune in comuni:
    try:
        # Try to get the geometry for the current comune in the province of Brescia
        comune_gdf = ox.geocode_to_gdf(f"{comune}, Brescia, Italy")
        comuni_polygons.append(comune_gdf.geometry.iloc[0])
    except Exception as e:
        # If geocoding fails, print the error and store None
        print(f"Geocoding error {comune}: {e}")
        comuni_polygons.append(None)

# Add the geometries to the DataFrame as a new column
df_finale['geometry'] = comuni_polygons

# Convert the DataFrame to a GeoDataFrame with EPSG:4326 (WGS84) coordinate system
gdf_comuni = gpd.GeoDataFrame(df_finale, geometry='geometry', crs="EPSG:4326")

### Fill each municipality with hexagons

Each municipality polygon is converted into a set of H3 hexagons. Each hexagon becomes a row in a new DataFrame, inheriting the municipality name and its current energy demand.
Then each hexagon ID is converted to a polygon geometry, creating a proper GeoDataFrame for spatial analysis.

In [ ]:
# Initialize an empty list to store the rows for the final expanded DataFrame
righe = []

# Loop through each row in the GeoDataFrame of municipalities
for idx, row in gdf_comuni.iterrows():
    # Proceed only if the geometry is not None
    if row['geometry'] is not None:
        # Get the list of H3 hexagon IDs that cover the geometry of the current municipality
        esagoni_ids = list(h3.polyfill_geojson(row['geometry'].__geo_interface__, resolution))
        # For each hexagon, create a dictionary with the municipality, demand, and H3 ID
        for h3_id in esagoni_ids:
            righe.append({
                'Comune': row['Comune'],
                'Domanda Giornaliera (kWh) - Attuali per esagono': row['Domanda Giornaliera (kWh) - Attuali per esagono'],
                'h3_id': h3_id
            })

# Create a DataFrame from the list of dictionaries
df_finale_expanded = pd.DataFrame(righe)

# Convert each H3 ID to a polygon and store it as geometry
df_finale_expanded['geometry'] = df_finale_expanded['h3_id'].apply(
    lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True))
)

# Convert the DataFrame into a GeoDataFrame with the specified coordinate reference system
gdf_finale_expanded = gpd.GeoDataFrame(df_finale_expanded, geometry='geometry', crs="EPSG:4326")

# Merge the altitude information from df_brescia based on the 'Comune' field
gdf_finale_expanded = gdf_finale_expanded.merge(df_brescia[['Comune', 'Altitudine']], on='Comune', how='left')

### Download the road network and keep main roads

The script retrieves the road network for the province and filters it to retain only major roads: primary, trunk, secondary, and tertiary.


In [ ]:
# Create a graph of streets within the Brescia polygon, focusing on roads suitable for driving
strade = ox.graph_from_polygon(brescia_polygon, network_type='drive')

# Convert the street network graph to a GeoDataFrame, excluding nodes
strade_gdf = ox.graph_to_gdfs(strade, nodes=False)

# Filter the streets to include only primary, trunk, secondary, and tertiary roads
strade_statali = strade_gdf[strade_gdf['highway'].isin(['primary', 'trunk', 'secondary', 'tertiary'])]

# Reproject the streets' GeoDataFrame to match the CRS of the expanded municipality GeoDataFrame
strade_statali = strade_statali.to_crs(gdf_finale_expanded.crs)

### Calculate total length of roads in each hexagon

For each hexagon, the script calculates the total length of intersecting roads and stores the value in a new column.

In [ ]:
# Initialize a new column in the GeoDataFrame to store the length of roads for each hexagon
gdf_finale_expanded['lunghezza_strade'] = 0.0

# Loop through each row in the expanded GeoDataFrame
for idx, hex_row in gdf_finale_expanded.iterrows():
    # Find the roads (strade_statali) that intersect with the current hexagon's geometry
    intersezioni = strade_statali[strade_statali.intersects(hex_row['geometry'])]

    # Calculate the total length of roads that intersect with the current hexagon
    lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()

    # Store the calculated total length in the 'lunghezza_strade' column for the current row
    gdf_finale_expanded.at[idx, 'lunghezza_strade'] = lunghezza_totale

Output streaming troncato alle ultime 5000 righe.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-36-1d32b048d920>:10: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-36-1d32b048d920>:10: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-36-1d32b048d920>:10: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(he

### Normalize road density

The length of roads is normalized between 0,5 and 1, giving more weight to hexagons with higher road coverage.


In [ ]:
# Calculate the minimum and maximum values for the 'lunghezza_strade' column
mean_lunghezza = gdf_finale_expanded['lunghezza_strade'].mean()
max_lunghezza = gdf_finale_expanded['lunghezza_strade'].max()

# Calculate the weight based on the length of the roads (normalized between 0.5 and 1)
gdf_finale_expanded['peso_lunghezza'] = 0.5 * ((gdf_finale_expanded['lunghezza_strade'] / mean_lunghezza) + (gdf_finale_expanded['lunghezza_strade'] / max_lunghezza))

### Count buildings in each hexagon

Buildings from OpenStreetMap are downloaded and intersected with each hexagon to count how many fall inside each area.

In [ ]:
# Print message indicating the start of building data download from OpenStreetMap
print("Scarico edifici da OpenStreetMap...")

# Download building data for the Brescia polygon from OpenStreetMap
edifici_gdf = ox.features_from_polygon(brescia_polygon, tags={"building": True})

# Reproject the building GeoDataFrame to match the CRS of the expanded municipalities GeoDataFrame
edifici_gdf = edifici_gdf.to_crs(gdf_finale_expanded.crs)

# Initialize a new column to store the number of buildings in each hexagon
gdf_finale_expanded['num_edifici'] = 0

# Loop through each row in the expanded GeoDataFrame of municipalities
for idx, hex_row in gdf_finale_expanded.iterrows():
    # Find the buildings that intersect with the current hexagon's geometry
    edifici_in_hex = edifici_gdf[edifici_gdf.intersects(hex_row['geometry'])]

    # Store the number of buildings in the 'num_edifici' column for the current hexagon
    gdf_finale_expanded.at[idx, 'num_edifici'] = len(edifici_in_hex)

Scarico edifici da OpenStreetMap...


### Normalize building density

The number of buildings is normalized between 0 and 1, giving higher weight to denser hexagons.


In [ ]:
# Calculate the maximum value in the 'num_edifici' column (number of buildings per hexagon)
max_edifici = gdf_finale_expanded['num_edifici'].max()

# Calculate the weight based on the number of buildings in each hexagon, normalized by the maximum value
gdf_finale_expanded['peso_edifici'] = gdf_finale_expanded['num_edifici'] / max_edifici

### Normalize altitude (penalize higher elevations)

Higher altitudes are penalized with lower scores. Normalization is scaled between 0.5 and 1.


In [ ]:
# Calculate the minimum and maximum values in the 'Altitudine' column
mean_altitudine = gdf_finale_expanded['Altitudine'].mean()
max_alt = gdf_finale_expanded['Altitudine'].max()

# Calculate the weight based on the altitude, normalized between 0.5 and 1 (higher altitudes have higher weights)
gdf_finale_expanded['peso_altitudine'] = (
    1 - 0.5 * (
        (gdf_finale_expanded['Altitudine'] / mean_altitudine) +
        (gdf_finale_expanded['Altitudine'] / max_alt)
    )
).clip(lower=0)


### Compute final weight for each hexagon

The final weight is the average of the normalized road length, building count, and adjusted altitude.


In [ ]:
# Calculate the final weight by averaging the weights for road length, number of buildings, and altitude
gdf_finale_expanded['peso_finale'] = (
    gdf_finale_expanded['peso_lunghezza'] +  # Weight based on road length
    gdf_finale_expanded['peso_edifici'] +   # Weight based on number of buildings
    gdf_finale_expanded['peso_altitudine']  # Weight based on altitude
) / 3  # Average the three weights to get the final weight

### Compute weighted daily energy demand

The raw energy demand is adjusted by the final weight, resulting in a refined estimate per hexagon.


In [ ]:
# Calculate the weighted daily demand (kWh) by multiplying the current daily demand with the final weight for each hexagon
gdf_finale_expanded['Domanda Giornaliera Ponderata (kWh)'] = (
    gdf_finale_expanded['Domanda Giornaliera (kWh) - Attuali per esagono'] *  # Current daily demand for each hexagon
    gdf_finale_expanded['peso_finale']  # Final weight calculated based on road length, buildings, and altitude
)

## Display the result

Prints a preview of the final dataset, showing key metrics and the computed weighted energy demand.


In [ ]:
# Print a subset of the GeoDataFrame showing relevant columns for the first few rows
print(gdf_finale_expanded[[  # Specify the columns to display
    'Comune',  # Name of the municipality
    'h3_id',  # H3 hexagon ID
    'lunghezza_strade',  # Length of roads in the hexagon
    'peso_lunghezza',  # Weight based on road length
    'num_edifici',  # Number of buildings in the hexagon
    'peso_edifici',  # Weight based on number of buildings
    'Altitudine',  # Altitude of the hexagon
    'peso_altitudine',  # Weight based on altitude
    'peso_finale',  # Final weight (average of the previous weights)
    'Domanda Giornaliera (kWh) - Attuali per esagono',  # Current daily demand (kWh) per hexagon
    'Domanda Giornaliera Ponderata (kWh)'  # Weighted daily demand (kWh) based on the final weight
]].head())  # Display the first few rows (default is 5)

    Comune            h3_id  lunghezza_strade  peso_lunghezza  num_edifici  \
0  brescia  881f99274bfffff          0.007774        0.453786           41   
1  brescia  881f992215fffff          0.013214        0.771287           55   
2  brescia  881f992213fffff          0.029853        1.742510          368   
3  brescia  881f9935c7fffff          0.000000        0.000000           37   
4  brescia  881f9935a1fffff          0.000000        0.000000            0   

   peso_edifici  Altitudine  peso_altitudine  peso_finale  \
0      0.037546         149         0.748972     0.413434   
1      0.050366         149         0.748972     0.523542   
2      0.336996         149         0.748972     0.942826   
3      0.033883         149         0.748972     0.260951   
4      0.000000         149         0.748972     0.249657   

   Domanda Giornaliera (kWh) - Attuali per esagono  \
0                                        71.409472   
1                                        71.409472   
2 

### Add the geometry of the hexagons to the DataFrame



In [ ]:
# Create a list of polygons from the H3 indices by converting each H3 index to a polygon geometry
poligoni_h3 = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in gdf_finale_expanded['h3_id']]

# Create a GeoDataFrame using the expanded DataFrame and the hexagon polygons, setting the coordinate reference system to EPSG:4326
gdf_finale = gpd.GeoDataFrame(gdf_finale_expanded, geometry=poligoni_h3, crs="EPSG:4326")

# Verify that the geometry has been added correctly by displaying the first few rows
print(gdf_finale.head())

    Comune  Domanda Giornaliera (kWh) - Attuali per esagono            h3_id  \
0  brescia                                        71.409472  881f99274bfffff   
1  brescia                                        71.409472  881f992215fffff   
2  brescia                                        71.409472  881f992213fffff   
3  brescia                                        71.409472  881f9935c7fffff   
4  brescia                                        71.409472  881f9935a1fffff   

                                            geometry  Altitudine  \
0  POLYGON ((10.29213 45.52556, 10.29032 45.52091...         149   
1  POLYGON ((10.16254 45.52223, 10.16074 45.51758...         149   
2  POLYGON ((10.18259 45.52669, 10.18079 45.52205...         149   
3  POLYGON ((10.26518 45.56623, 10.26337 45.56159...         149   
4  POLYGON ((10.26591 45.53671, 10.2641 45.53207,...         149   

   lunghezza_strade  peso_lunghezza  num_edifici  peso_edifici  \
0          0.007774        0.453786         

## Save gpkg

In [ ]:
# Save the aggregated GeoDataFrame to a GeoPackage file
gdf_finale.to_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_finale_ind.gpkg", layer='layer_name', driver='GPKG')

## Visualization of Weighted Daily Electricity Demand with Heatmaps

### Color Map Legend and Hexagonal Polygon Visualization

In [ ]:
valori_validi = gdf_finale[colonna_heatmap].dropna()

# Calcola min e max in modo sicuro
min_val = valori_validi.min()
max_val = valori_validi.max()

print(min_val)

print(max_val)

0.0
263.42859326708793


In [ ]:
import folium  # For creating interactive maps
import branca.colormap as cm  # For color mapping
import pandas as pd  # For handling data
gdf_finale = gdf_finale.to_crs(epsg=4326)
# Select the column to visualize, in this case, the weighted daily demand
colonna_heatmap = "Domanda Giornaliera Ponderata (kWh)"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)  # Coordinates of Brescia

# Create a continuous color map based on the min and max values of the selected column
min_val = gdf_finale[colonna_heatmap].min()  # Minimum value of the column
max_val = gdf_finale[colonna_heatmap].max()  # Maximum value of the column
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)  # Create color scale from yellow to red
colormap.caption = f"{colonna_heatmap}"  # Add caption to the color map

# Add the hexagons with color based on the "Domanda Giornaliera Ponderata" variable
for _, row in gdf_finale.iterrows():
    valore = row[colonna_heatmap]  # Get the value for each hexagon
    colore = colormap(valore) if not pd.isna(valore) else "#cccccc"  # Assign color based on value, gray for NaN

    # Use the geometry (geometry column of the GeoDataFrame) for each hexagon
    esagono = row['geometry']

    # Create a polygon based on the geometry and coordinates
    folium.Polygon(
        locations=[(y, x) for x, y in esagono.exterior.coords],  # Extract coordinates from polygon geometry
        fill=True,
        fill_opacity=0.7,  # Set the opacity of the fill color
        color=None,
        fill_color=colore,  # Set the fill color based on the value
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore, 2)}"  # Display a popup with the municipality and value
    ).add_to(mappa)

# Add the color map legend to the map
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

### Log-Scale Heatmap Visualization for Daily Demand

In [ ]:
import folium  # For creating interactive maps
import branca.colormap as cm  # For color mapping
import pandas as pd  # For handling data
import numpy as np  # <-- Import numpy for the log scale

# Select the column to visualize
colonna_heatmap = "Domanda Giornaliera Ponderata (kWh)"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)  # Coordinates of Brescia

# Calculate the min and max values using the log scale (adding +1 to avoid log(0))
min_val = np.log1p(gdf_finale[colonna_heatmap]).min()  # Minimum value of the column after log(1+x)
max_val = np.log1p(gdf_finale[colonna_heatmap]).max()  # Maximum value of the column after log(1+x)

# Create a continuous color map between the min and max values using the log scale
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)  # Create color scale from yellow to red
colormap.caption = f"{colonna_heatmap} (log scale)"  # Add caption to the color map

# Add the hexagons with color based on the "Domanda Giornaliera Ponderata" variable (log-scaled)
for _, row in gdf_finale.iterrows():
    valore_originale = row[colonna_heatmap]  # Get the original value for each hexagon
    valore_log = np.log1p(valore_originale)  # log(1 + value) to avoid issues with zero

    colore = colormap(valore_log) if not pd.isna(valore_log) else "#cccccc"  # Assign color based on log value

    # Use the geometry (geometry column of the GeoDataFrame) for each hexagon
    esagono = row['geometry']

    # Create a polygon based on the geometry and coordinates
    folium.Polygon(
        locations=[(y, x) for x, y in esagono.exterior.coords],  # Extract coordinates from polygon geometry
        fill=True,
        fill_opacity=0.7,  # Set the opacity of the fill color
        color=None,
        fill_color=colore,  # Set the fill color based on the log-transformed value
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore_originale, 2)}"  # Display a popup with the municipality and original value
    ).add_to(mappa)

# Add the color map legend to the map
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/log_heatmap_idx_current_scenario_ind.html")

## Aggregation of Energy Demand at Municipality Level

In [ ]:
# Load the file if needed
gdf_finale.to_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_finale_ind.gpkg", layer='layer_name')

# Remove the 'h3_id' column since we don't need it anymore
gdf_finale = gdf_finale.drop(columns=['h3_id'])

# Now perform a groupby by municipality, summing all other numerical columns
gdf_comuni = gdf_finale.groupby('Comune').agg({
    'Domanda Giornaliera (kWh) - Attuali per esagono': 'sum',
    #'lunghezza_strade': 'sum',
    #'peso_lunghezza': 'sum',
    'Domanda Giornaliera Ponderata (kWh)': 'sum',
    'geometry': 'first'  # take the geometry of the municipality (they are identical within the same municipality)
}).reset_index()

# Recreate a GeoDataFrame
gdf_comuni = gpd.GeoDataFrame(gdf_comuni, geometry='geometry', crs=gdf_finale.crs)

# Save to file if desired
#gdf_comuni.to_file('gdf_comuni_aggregato.gpkg', driver='GPKG')

# Display the result
print(gdf_comuni.head())

        Comune  Domanda Giornaliera (kWh) - Attuali per esagono  \
0  acquafredda                                        66.649693   
1         adro                                       354.287772   
2     agnosine                                        92.444721   
3   alfianello                                        83.381337   
4         anfo                                        25.714062   

   Domanda Giornaliera Ponderata (kWh)  \
0                            34.314705   
1                           166.156393   
2                            32.782986   
3                            41.036736   
4                             5.383388   

                                            geometry  
0  POLYGON ((10.41497 45.32347, 10.41316 45.31882...  
1  POLYGON ((9.92957 45.61423, 9.92778 45.60959, ...  
2  POLYGON ((10.3557 45.64132, 10.35388 45.63668,...  
3  POLYGON ((10.14331 45.26738, 10.14152 45.26273...  
4  POLYGON ((10.49944 45.80407, 10.49762 45.79944...  


## Save gpkg

In [ ]:
# Save the aggregated GeoDataFrame to a GeoPackage file
gdf_comuni.to_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_comuni_ind.gpkg", layer='layer_name', driver='GPKG')

# Projection to 2030 - Conservative Scenario

## Load and Display Merged Dataset

In [ ]:
import pandas as pd

# Load the CSV file containing the 2030 conservative scenario data for hexagons
df_conservativo = pd.read_csv("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/df_2030_conservativo_esagoni.csv")

## Loading a Brescia province dataset for specific columns

In [ ]:
# Load the CSV file containing data for municipalities in Brescia
df_brescia = pd.read_csv("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/df_brescia.csv")

# Clean the 'Comune' column: remove leading/trailing spaces and convert to lowercase
df_brescia['Comune'] = df_brescia['Comune'].str.strip().str.lower()

# Display the first few rows of the DataFrame
df_brescia.head()

,Comune,Popolazione,Superficie,Densità,Altitudine
0,brescia,199949,90.58,2207.0,149
1,desenzano del garda,29275,59.19,495.0,67
2,montichiari,26372,82.02,322.0,104
3,lumezzane,21609,31.39,689.0,460
4,palazzolo sull'oglio,20390,23.31,875.0,166


# Creating indices

## Importing libraries and setting resolution

This step imports necessary Python libraries and also sets the H3 resolution level to 8 (moderate granularity).

In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import h3
from shapely.geometry import Polygon

# Set the resolution level for H3 hexagons (higher value = finer grid)
resolution = 8

### Load boundaries and municipalities

In [ ]:
# Get the polygon of the province of Brescia
provincia_gdf = ox.geocode_to_gdf("Brescia, Italy")
brescia_polygon = provincia_gdf.geometry.iloc[0]

# GeoDataFrame of municipalities
comuni = df_conservativo['Comune'].tolist()
comuni_polygons = []

# Retrieve the polygons of each municipality
for comune in comuni:
    try:
        comune_gdf = ox.geocode_to_gdf(f"{comune}, Brescia, Italy")
        comuni_polygons.append(comune_gdf.geometry.iloc[0])
    except Exception as e:
        print(f"Error geocoding {comune}: {e}")
        comuni_polygons.append(None)

# Add geometry to df_conservativo
df_conservativo['geometry'] = comuni_polygons
gdf_comuni = gpd.GeoDataFrame(df_conservativo, geometry='geometry', crs="EPSG:4326")

### Fill each municipality with hexagons

Each municipality polygon is converted into a set of H3 hexagons. Each hexagon becomes a row in a new DataFrame, inheriting the municipality name and its current energy demand.
Then each hexagon ID is converted to a polygon geometry, creating a proper GeoDataFrame for spatial analysis.

In [ ]:
# Expand: each hexagon = one row
righe = []
for idx, row in gdf_comuni.iterrows():
    if row['geometry'] is not None:
        esagoni_ids = list(h3.polyfill_geojson(row['geometry'].__geo_interface__, resolution))
        for h3_id in esagoni_ids:
            righe.append({
                'Comune': row['Comune'],
                'Domanda Giornaliera (kWh) - Conservativo': row['Domanda Giornaliera (kWh) - Conservativo'],
                'h3_id': h3_id
            })

# Create the expanded DataFrame
df_conservativo_expanded = pd.DataFrame(righe)

# Add hexagon geometry
df_conservativo_expanded['geometry'] = df_conservativo_expanded['h3_id'].apply(
    lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True))
)
gdf_conservativo_expanded = gpd.GeoDataFrame(df_conservativo_expanded, geometry='geometry', crs="EPSG:4326")

# LEFT JOIN with df_brescia (altitude information)
gdf_conservativo_expanded = gdf_conservativo_expanded.merge(df_brescia[['Comune', 'Altitudine']], on='Comune', how='left')

### Download the road network and keep main roads

The script retrieves the road network for the province and filters it to retain only major roads: primary, trunk, secondary, and tertiary.

In [ ]:
# Download the road network
strade = ox.graph_from_polygon(brescia_polygon, network_type='drive')
strade_gdf = ox.graph_to_gdfs(strade, nodes=False)

# Filter for main roads
strade_statali = strade_gdf[strade_gdf['highway'].isin(['primary', 'trunk', 'secondary', 'tertiary'])]

# Ensure the coordinate reference system is the same
strade_statali = strade_statali.to_crs(gdf_conservativo_expanded.crs)

### Calculate total length of roads in each hexagon

For each hexagon, the script calculates the total length of intersecting roads and stores the value in a new column.

In [ ]:
# Calculate the total length of main roads within each hexagon
gdf_conservativo_expanded['lunghezza_strade'] = 0.0

for idx, hex_row in gdf_conservativo_expanded.iterrows():
    intersezioni = strade_statali[strade_statali.intersects(hex_row['geometry'])]
    lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
    gdf_conservativo_expanded.at[idx, 'lunghezza_strade'] = lunghezza_totale

Output streaming troncato alle ultime 5000 righe.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-72-82148c2e9fa1>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-72-82148c2e9fa1>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-72-82148c2e9fa1>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_r

### Normalize road density

The length of roads is normalized between 0,5 and 1, giving more weight to hexagons with higher road coverage.


In [ ]:
# Calculate the minimum and maximum values for the 'lunghezza_strade' column
mean_lunghezza = gdf_conservativo_expanded['lunghezza_strade'].mean()
max_lunghezza = gdf_conservativo_expanded['lunghezza_strade'].max()

# Calculate the weight based on the length of the roads (normalized between 0.5 and 1)
gdf_conservativo_expanded['peso_lunghezza'] = 0.5 * ((gdf_conservativo_expanded['lunghezza_strade'] / mean_lunghezza) + (gdf_conservativo_expanded['lunghezza_strade'] / max_lunghezza))

### Normalize altitude (penalize higher elevations)

Higher altitudes are penalized with lower scores. Normalization is scaled between 0.5 and 1.

In [ ]:
# Calculate the altitude index
mean_altitudine = gdf_conservativo_expanded['Altitudine'].mean()
max_alt = gdf_conservativo_expanded['Altitudine'].max()

# Invert altitude to favor lower elevations
gdf_conservativo_expanded['peso_altitudine'] = (
    1 - 0.5 * (
        (gdf_conservativo_expanded['Altitudine'] / mean_altitudine) +
        (gdf_conservativo_expanded['Altitudine'] / max_alt)
    )
).clip(lower=0)

### Compute final weight for each hexagon

The final weight is the average of the normalized road length, building count, and adjusted altitude.

In [ ]:
# Calculate the final weight (average of the two weights)
gdf_conservativo_expanded['peso_finale'] = (
    gdf_conservativo_expanded['peso_lunghezza'] +
    gdf_conservativo_expanded['peso_altitudine']
    ) / 2

### Compute weighted daily energy demand

The raw energy demand is adjusted by the final weight, resulting in a refined estimate per hexagon.

In [ ]:
# Calculate the final weighted daily demand
gdf_conservativo_expanded['Domanda Giornaliera Ponderata (kWh)'] = (
    gdf_conservativo_expanded['Domanda Giornaliera (kWh) - Conservativo'] *
    gdf_conservativo_expanded['peso_finale']
)

## Display the result

Prints a preview of the final dataset, showing key metrics and the computed weighted energy demand.

In [ ]:
# Final output
print(gdf_conservativo_expanded[[
    'Comune',
    'h3_id',
    'lunghezza_strade',
    'peso_lunghezza',
    'Altitudine',
    'peso_altitudine',
    'peso_finale',
    'Domanda Giornaliera (kWh) - Conservativo',
    'Domanda Giornaliera Ponderata (kWh)'
]].head())

    Comune            h3_id  lunghezza_strade  peso_lunghezza  Altitudine  \
0  brescia  881f99274bfffff          0.007774        0.453786         149   
1  brescia  881f992215fffff          0.013214        0.771287         149   
2  brescia  881f992213fffff          0.029853        1.742510         149   
3  brescia  881f9935c7fffff          0.000000        0.000000         149   
4  brescia  881f9935a1fffff          0.000000        0.000000         149   

   peso_altitudine  peso_finale  Domanda Giornaliera (kWh) - Conservativo  \
0         0.748972     0.601379                              37132.881058   
1         0.748972     0.760129                              37132.881058   
2         0.748972     1.245741                              37132.881058   
3         0.748972     0.374486                              37132.881058   
4         0.748972     0.374486                              37132.881058   

   Domanda Giornaliera Ponderata (kWh)  
0                         22330.9

### Add the geometry of the hexagons to the DataFrame

In [ ]:
# Create a list of polygons from H3 indices
poligoni_h3 = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in gdf_conservativo_expanded['h3_id']]

# Create a GeoDataFrame with the geometry
gdf_conservativo = gpd.GeoDataFrame(gdf_conservativo_expanded, geometry=poligoni_h3, crs="EPSG:4326")

# Check if the geometry has been added correctly
print(gdf_conservativo.head())

    Comune  Domanda Giornaliera (kWh) - Conservativo            h3_id  \
0  brescia                              37132.881058  881f99274bfffff   
1  brescia                              37132.881058  881f992215fffff   
2  brescia                              37132.881058  881f992213fffff   
3  brescia                              37132.881058  881f9935c7fffff   
4  brescia                              37132.881058  881f9935a1fffff   

                                            geometry  Altitudine  \
0  POLYGON ((10.29213 45.52556, 10.29032 45.52091...         149   
1  POLYGON ((10.16254 45.52223, 10.16074 45.51758...         149   
2  POLYGON ((10.18259 45.52669, 10.18079 45.52205...         149   
3  POLYGON ((10.26518 45.56623, 10.26337 45.56159...         149   
4  POLYGON ((10.26591 45.53671, 10.2641 45.53207,...         149   

   lunghezza_strade  peso_lunghezza  peso_altitudine  peso_finale  \
0          0.007774        0.453786         0.748972     0.601379   
1          0.0

## Save gpkg

In [ ]:
# Save the aggregated GeoDataFrame to a GeoPackage file
gdf_conservativo.to_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_conservativo_finale_ind.gpkg", layer='layer_name', driver='GPKG')

In [ ]:
import geopandas as gpd
# Load the GeoDataFrame for the conservative 2030 scenario
gdf_conservativo = gpd.read_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_conservativo_finale_ind.gpkg")

## Visualization of Weighted Daily Electricity Demand with Heatmaps

### Log-Scale Heatmap Visualization for Daily Demand

In [ ]:
import folium
import branca.colormap as cm
import pandas as pd
import numpy as np  # <-- Import numpy for log

# Select the column to visualize
colonna_heatmap = "Domanda Giornaliera Ponderata (kWh)"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Calculate min and max on the log scale (adding +1 to avoid log(0))
min_val = np.log1p(gdf_conservativo[colonna_heatmap]).min()
max_val = np.log1p(gdf_conservativo[colonna_heatmap]).max()

# Create the continuous color map between min and max on a log scale
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = f"{colonna_heatmap} (log scale)"

# Add hexagons with coloring based on the variable "Domanda Giornaliera Ponderata" (log-scaled)
for _, row in gdf_conservativo.iterrows():
    valore_originale = row[colonna_heatmap]
    valore_log = np.log1p(valore_originale)  # log(1 + value) to avoid errors with zero

    colore = colormap(valore_log) if not pd.isna(valore_log) else "#cccccc"

    # Use the geometry (column 'geometry' in the GeoDataFrame) for each hexagon
    esagono = row['geometry']

    folium.Polygon(
        locations=[(y, x) for x, y in esagono.exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore_originale, 2)}"
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/log_heatmap_idx_conservative_scenario_ind.html")

# Projection to 2030 - Optimistic Scenario

## Load and Display Merged Dataset

In [ ]:
import pandas as pd

# Load the CSV file containing the 2030 conservative scenario data for hexagons
# df_conservativo = pd.read_csv('df_2030_conservativo_esagoni.csv')
df_ottimistico = pd.read_csv("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/02 - Esagoni/df_2030_ottimistico_esagoni.csv")

## Loading a Brescia province dataset for specific columns

In [ ]:
# Load the CSV file containing data for municipalities in Brescia
df_brescia = pd.read_csv("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/df_brescia.csv")

# Clean the 'Comune' column: remove leading/trailing spaces and convert to lowercase
df_brescia['Comune'] = df_brescia['Comune'].str.strip().str.lower()

# Display the first few rows of the DataFrame
df_brescia.head()

,Comune,Popolazione,Superficie,Densità,Altitudine
0,brescia,199949,90.58,2207.0,149
1,desenzano del garda,29275,59.19,495.0,67
2,montichiari,26372,82.02,322.0,104
3,lumezzane,21609,31.39,689.0,460
4,palazzolo sull'oglio,20390,23.31,875.0,166


# Creating indices

## Importing libraries and setting resolution

This step imports necessary Python libraries and also sets the H3 resolution level to 8 (moderate granularity).

In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox
import h3
from shapely.geometry import Polygon

# Set the resolution level for H3 hexagons (higher value = finer grid)
resolution = 8

### Load boundaries and municipalities

In [ ]:
# Get the polygon of the province of Brescia
provincia_gdf = ox.geocode_to_gdf("Brescia, Italy")
brescia_polygon = provincia_gdf.geometry.iloc[0]

# GeoDataFrame of the municipalities
comuni = df_ottimistico['Comune'].tolist()
comuni_polygons = []

# Get the polygons of the municipalities
for comune in comuni:
    try:
        comune_gdf = ox.geocode_to_gdf(f"{comune}, Brescia, Italy")
        comuni_polygons.append(comune_gdf.geometry.iloc[0])
    except Exception as e:
        print(f"Error geocoding {comune}: {e}")
        comuni_polygons.append(None)

# Add geometry to df_ottimistico
df_ottimistico['geometry'] = comuni_polygons
gdf_comuni = gpd.GeoDataFrame(df_ottimistico, geometry='geometry', crs="EPSG:4326")

### Fill each municipality with hexagons

Each municipality polygon is converted into a set of H3 hexagons. Each hexagon becomes a row in a new DataFrame, inheriting the municipality name and its current energy demand.
Then each hexagon ID is converted to a polygon geometry, creating a proper GeoDataFrame for spatial analysis.

In [ ]:
# Expand: each hexagon = one row
righe = []
for idx, row in gdf_comuni.iterrows():
    if row['geometry'] is not None:
        esagoni_ids = list(h3.polyfill_geojson(row['geometry'].__geo_interface__, resolution))
        for h3_id in esagoni_ids:
            righe.append({
                'Comune': row['Comune'],
                'Domanda Giornaliera (kWh) - Ottimistico': row['Domanda Giornaliera (kWh) - Ottimistico'],
                'h3_id': h3_id
            })

# Create the expanded DataFrame
df_ottimistico_expanded = pd.DataFrame(righe)

# Add hexagon geometries
df_ottimistico_expanded['geometry'] = df_ottimistico_expanded['h3_id'].apply(
    lambda x: Polygon(h3.h3_to_geo_boundary(x, geo_json=True))
)
gdf_ottimistico_expanded = gpd.GeoDataFrame(df_ottimistico_expanded, geometry='geometry', crs="EPSG:4326")

# LEFT JOIN with df_brescia (altitudes)
gdf_ottimistico_expanded = gdf_ottimistico_expanded.merge(df_brescia[['Comune', 'Altitudine']], on='Comune', how='left')

### Download the road network and keep main roads

The script retrieves the road network for the province and filters it to retain only major roads: primary, trunk, secondary, and tertiary.

In [ ]:
# Download the road network
strade = ox.graph_from_polygon(brescia_polygon, network_type='drive')
strade_gdf = ox.graph_to_gdfs(strade, nodes=False)

# Filter main roads
strade_statali = strade_gdf[strade_gdf['highway'].isin(['primary', 'trunk', 'secondary', 'tertiary'])]

# Ensure the coordinate reference system is the same
strade_statali = strade_statali.to_crs(gdf_ottimistico_expanded.crs)

### Calculate total length of roads in each hexagon

For each hexagon, the script calculates the total length of intersecting roads and stores the value in a new column.

In [ ]:
# Calculate the length of roads per hexagon
gdf_ottimistico_expanded['lunghezza_strade'] = 0.0

for idx, hex_row in gdf_ottimistico_expanded.iterrows():
    intersezioni = strade_statali[strade_statali.intersects(hex_row['geometry'])]
    lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
    gdf_ottimistico_expanded.at[idx, 'lunghezza_strade'] = lunghezza_totale

Output streaming troncato alle ultime 5000 righe.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-89-44714e226a1c>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-89-44714e226a1c>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_row['geometry']).length.sum()
<ipython-input-89-44714e226a1c>:6: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  lunghezza_totale = intersezioni.intersection(hex_r

### Normalize road density

The length of roads is normalized between 0,5 and 1, giving more weight to hexagons with higher road coverage.


In [ ]:
# Calculate the minimum and maximum values for the 'lunghezza_strade' column
mean_lunghezza = gdf_ottimistico_expanded['lunghezza_strade'].mean()
max_lunghezza = gdf_ottimistico_expanded['lunghezza_strade'].max()

# Calculate the weight based on the length of the roads (normalized between 0.5 and 1)
gdf_ottimistico_expanded['peso_lunghezza'] = 0.5 * ((gdf_ottimistico_expanded['lunghezza_strade'] / mean_lunghezza) + (gdf_ottimistico_expanded['lunghezza_strade'] / max_lunghezza))



### Normalize altitude (penalize higher elevations)

Higher altitudes are penalized with lower scores. Normalization is scaled between 0.5 and 1.

In [ ]:
# Calculate the altitude index
mean_altitudine = gdf_ottimistico_expanded['Altitudine'].mean()
max_alt = gdf_ottimistico_expanded['Altitudine'].max()

# Invert altitude to favor lower elevations
gdf_ottimistico_expanded['peso_altitudine'] = (
    1 - 0.5 * (
        (gdf_ottimistico_expanded['Altitudine'] / mean_altitudine) +
        (gdf_ottimistico_expanded['Altitudine'] / max_alt)
    )
).clip(lower=0)

### Compute final weight for each hexagon

The final weight is the average of the normalized road length, building count, and adjusted altitude.

In [ ]:
# Calculate final weight (average of the 2 weights)
gdf_ottimistico_expanded['peso_finale'] = (
    gdf_ottimistico_expanded['peso_lunghezza'] +
    gdf_ottimistico_expanded['peso_altitudine']
) / 2

### Compute weighted daily energy demand

The raw energy demand is adjusted by the final weight, resulting in a refined estimate per hexagon.

In [ ]:
# Calculate final weighted daily demand
gdf_ottimistico_expanded['Domanda Giornaliera Ponderata (kWh)'] = (
    gdf_ottimistico_expanded['Domanda Giornaliera (kWh) - Ottimistico'] *
    gdf_ottimistico_expanded['peso_finale']
)

## Display the result

Prints a preview of the final dataset, showing key metrics and the computed weighted energy demand.

In [ ]:
# Final output
print(gdf_ottimistico_expanded[[
    'Comune',
    'h3_id',
    'lunghezza_strade',
    'peso_lunghezza',
    'Altitudine',
    'peso_altitudine',
    'peso_finale',
    'Domanda Giornaliera (kWh) - Ottimistico',
    'Domanda Giornaliera Ponderata (kWh)'
]].head())

    Comune            h3_id  lunghezza_strade  peso_lunghezza  Altitudine  \
0  brescia  881f99274bfffff          0.007774        0.453786         149   
1  brescia  881f992215fffff          0.013214        0.771287         149   
2  brescia  881f992213fffff          0.029853        1.742510         149   
3  brescia  881f9935c7fffff          0.000000        0.000000         149   
4  brescia  881f9935a1fffff          0.000000        0.000000         149   

   peso_altitudine  peso_finale  Domanda Giornaliera (kWh) - Ottimistico  \
0         0.748972     0.601379                             55699.511659   
1         0.748972     0.760129                             55699.511659   
2         0.748972     1.245741                             55699.511659   
3         0.748972     0.374486                             55699.511659   
4         0.748972     0.374486                             55699.511659   

   Domanda Giornaliera Ponderata (kWh)  
0                         33496.501242 

### Add the geometry of the hexagons to the DataFrame

In [ ]:
# Create a list of polygons from H3 indices
poligoni_h3 = [Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in gdf_ottimistico_expanded['h3_id']]

# Create a GeoDataFrame with the geometry
gdf_ottimistico = gpd.GeoDataFrame(gdf_ottimistico_expanded, geometry=poligoni_h3, crs="EPSG:4326")

# Check that the geometry was added correctly
print(gdf_ottimistico.head())

    Comune  Domanda Giornaliera (kWh) - Ottimistico            h3_id  \
0  brescia                             55699.511659  881f99274bfffff   
1  brescia                             55699.511659  881f992215fffff   
2  brescia                             55699.511659  881f992213fffff   
3  brescia                             55699.511659  881f9935c7fffff   
4  brescia                             55699.511659  881f9935a1fffff   

                                            geometry  Altitudine  \
0  POLYGON ((10.29213 45.52556, 10.29032 45.52091...         149   
1  POLYGON ((10.16254 45.52223, 10.16074 45.51758...         149   
2  POLYGON ((10.18259 45.52669, 10.18079 45.52205...         149   
3  POLYGON ((10.26518 45.56623, 10.26337 45.56159...         149   
4  POLYGON ((10.26591 45.53671, 10.2641 45.53207,...         149   

   lunghezza_strade  peso_lunghezza  peso_altitudine  peso_finale  \
0          0.007774        0.453786         0.748972     0.601379   
1          0.013214 

## Save gpkg

In [ ]:
# Save the aggregated GeoDataFrame to a GeoPackage file
gdf_ottimistico.to_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_ottimistico_finale_ind.gpkg", layer='layer_name', driver='GPKG')

In [ ]:
import geopandas as gpd
# Load the GeoDataFrame for the conservative 2030 scenario
gdf_ottimistico= gpd.read_file("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/gdf_2030_ottimistico_finale_ind.gpkg")

## Visualization of Weighted Daily Electricity Demand with Heatmaps

### Log-Scale Heatmap Visualization for Daily Demand

In [ ]:
import folium
import branca.colormap as cm
import pandas as pd
import numpy as np  # <-- Import numpy for log transformation

# Select the column to display
colonna_heatmap = "Domanda Giornaliera Ponderata (kWh)"

# Create the map centered on Brescia
mappa = folium.Map(location=[45.54, 10.22], zoom_start=10)

# Calculate min and max on the log scale (adding +1 to avoid log(0))
min_val = np.log1p(gdf_ottimistico[colonna_heatmap]).min()
max_val = np.log1p(gdf_ottimistico[colonna_heatmap]).max()

# Create the continuous colormap between min and max on the log scale
colormap = cm.linear.YlOrRd_09.scale(min_val, max_val)
colormap.caption = f"{colonna_heatmap} (log scale)"

# Add hexagons with coloring based on the "Domanda Giornaliera Ponderata" (log-scaled)
for _, row in gdf_ottimistico.iterrows():
    valore_originale = row[colonna_heatmap]
    valore_log = np.log1p(valore_originale)  # log(1 + value) to avoid errors on zero

    colore = colormap(valore_log) if not pd.isna(valore_log) else "#cccccc"

    # Use the geometry (the 'geometry' column of the GeoDataFrame) for each hexagon
    esagono = row['geometry']

    folium.Polygon(
        locations=[(y, x) for x, y in esagono.exterior.coords],
        fill=True,
        fill_opacity=0.7,
        color=None,
        fill_color=colore,
        weight=0.2,
        popup=f"{row['Comune']}<br>{colonna_heatmap}: {round(valore_originale, 2)}"
    ).add_to(mappa)

# Add the legend
colormap.add_to(mappa)

# Display the map
mappa

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# Save the map as an HTML file
mappa.save("/content/drive/Shareddrives/OM in Business Analytics/OM in BA - Project/Codice/03 - Indici/log_heatmap_idx_optimistic_scenario_ind.html")